### Import libraries

In [1]:
import sys
import numpy as np
import os
import pandas as pd
import uproot
import time
import matplotlib.pyplot as plt
import awkward as ak
import matplotlib as mpl



from dt5202 import dt5202_crunch # Import parser that Tom wrote. This is containted in the file 
                                 # named dt5202.py in the lib folder.
from dt5202oldDAQ import dt5202_crunch as dt5202_crunch_oldDAQ # Old ver

from langaus import LanGausFit 


import warnings
warnings.filterwarnings("ignore")

Welcome to JupyROOT 6.24/02


### Import Simulation Data (Muons)

In [2]:
folder = "/home/jat89/project/muonVeto/muonFiles/"

colnames = ["ChainNumber", "ParticleInputEnergy", "ParticleDirectionX", "ParticleDirectionY", "ParticleDirectionZ", "Energy_Phase0BottomMVThickSolid"]
array_muons = uproot.concatenate(folder+"Muons_25*.root:qtree", colnames, library = 'np', allow_missing = True)#question mark wildcard

df_muons = pd.DataFrame(array_muons)

### Import Simulation Data (Gamma)

In [3]:
folder = "/home/jat89/project/muonVeto/gammaFiles/phase0/"

colnames = ["ChainNumber", "ParticleInputEnergy", "ParticleDirectionX", "ParticleDirectionY"
            , "ParticleDirectionZ", "Energy_Phase0BottomMVThickSolid"]
array = uproot.concatenate(folder+"Gammas_5*.root:qtree", colnames, library = 'np', allow_missing = True)#question mark wildcard

df = pd.DataFrame(array)



### Put Gamma and Muon Data into Separate Arrays

In [4]:
gamma_spectrum = []
muon_spectrum = []
for element in df.Energy_Phase0BottomMVThickSolid:
    if(len(element)==0):
        continue
    else:
        gamma_spectrum.append(element[0])
        
for element in df_muons.Energy_Phase0BottomMVThickSolid:
    if(len(element)==0):
        continue
    else:
        muon_spectrum.append(element[0])

### Import Lab Data 

In [5]:
%%capture

sourceFolder = "/home/dt592/data/board1771_01192023_retestingSwirlyPanel/"

onlyfiles = [f for f in os.listdir(sourceFolder) if '.dat' in f]

data_list = []
for file in onlyfiles:
    data_list.append(dt5202_crunch(str(sourceFolder)+str(file), num_ev=100000000, num_ch=1))

data = data_list[0]
for i in range(1, len(data_list)):
    data = np.append(data, data_list[i])

#Declare constants
ped = 50
adc_mev = 538.3933387436877

# Convert lab data to MeV
d = data['data']['low_gain']-ped
lab_mev = (d.astype('float64'))/adc_mev # lab data in MeV

In [6]:
len(lab_mev)

223068